This ipynb shows an example (credited to CNPC) of using `get_1well` API to design a *complex* single well handling anticollision constraint.  
Here the *complex* means that rather than 2 turns in the well trajectory, there can be at most 3 turns:  
1st is the buildup turn after KOP.  
2nd is the control turn in the middle to fulfill complex constraints.  
3rd is the landing turn before the target(or the landing point).

This doesn't mean the well trajectory must have 3 turns. It automatically decides how many turns are needed based on the input parameters.

# 0. Initiate environment

In [ ]:
import requests
import json
import numpy as np
import pandas as pd
from plotly import graph_objects as go
import time

In [2]:
import sys
import os
rootpath = os.path.join(os.getcwd().split("WelLayout_API") [0], 
                        "WelLayout_API")
# print(rootpath)
sys.path.append(rootpath)

from API import get_1well
from tools.input2json import input2json
from tools.PlotSurvey_plotly import PlotSurvey
from tools.PlotContour_plotly import PlotContour, PlotDangerArea
from tools.PlotArrow_plotly import PlotArrow
import copy

# 1. Prepare Data

## 1.1 Target information
The completion intervals (targets) are normally decided by the reservoir engineers based on the reservoir layer's trend to maximize the total production of the wells.  
Here we need the location of the first point of the completion interval($PT$) and its desired entry direction($VT$) for each well.

Following is the data for 1 target.

In [3]:
# You can simple give the information of the well for design.
n=1

# location of the first point of the completion interval (PT), [X, Y, Z]
PTM= np.array ([[-503.27, -7075.19, -3150.74]]) # remember to make it as a 2D array

# drilling direction at the first point of the completion interval (VT),
VTM= np.array ([[8.17, -32.92, -0.11]])

## 1.2 KOP information
The KOP depth is normally determined by the drilling engineer based on the rock mechanical property of the formation to ensure the wellbore stabilty.  
Normally, the direction at KOP is vertically downwards.

For a single well trajectory design, the KOP location must be completely provided.

In [4]:
# location of the KOP (PK), [X, Y, Z]
# KOP is beneath site
PKM= np.array ([[-1336.47, -4518.5, -386.6]])

# drilling direction at the KOP (VK), [X, Y, Z]
VKM= np.array ([[0.0, 0.0, -1.0]])

## 1.3 DLS constraint
Dogleg severity (DLS) is constrained by the drilling equipment.  
Drilling engineers need to set a maximum allowable DLS to ensure the feasibility for drilling and the wellbore stability.  
DLS unit is given in `°/30m`

For the single well trajectory design, we can accept at most 3 curved sections:  
1st is the buildup turn after KOP.  
2nd is the control turn in the middle to fulfill complex constraints.  
3rd is the landing turn before the target(or the landing point).  

In [5]:
# DLS at the buildup section after the KOP 
dls_KOP=2.0

# DLS at the control section in the middle
dls_Control=2.5

# DLS at the landing section before the Target
dls_Target=3.0

# format DLS for API
DLSM= np.array ([[dls_KOP, dls_Control, dls_Target]])

# optional: get the corresponding curvature radius of the DLS
rM= 30*180/DLSM/np.pi
print(rM)

[[859.4366927  687.54935416 572.95779513]]


## 1.4 Anticollision constraints

### 1.4.1 load and check the existing offset wells' data

In [6]:
filepath_offset='survey_data.json'
with open(filepath_offset) as json_file:
    input_data = json.load(json_file)

Visualize the existing wells

In [7]:
fig=None
for well in range(len(input_data["SURVEY"])):
    fig=PlotSurvey(input_data["SURVEY"][well], fig, show=0)
     
fig.show()

show the information of the planning well

In [8]:
# # plot the PK and PT points in the fig
# fig.add_trace(go.Scatter3d(x=PKM[:,0], y=PKM[:,1], z=PKM[:,2], 
#                            name='KOP',
#                            mode='markers',
#                            marker=dict(size=3, color='blue')))
# fig.add_trace(go.Scatter3d(x=PTM[:,0], y=PTM[:,1], z=PTM[:,2], 
#                            name='Target',
#                            mode='markers',
#                            marker=dict(size=3, color='magenta')))

# plot an arrow of the VT, start at PT
fig=PlotArrow(start=PKM[0,:], vector=VKM[0,:]*300, fig=fig, style='b-', head_size=0.5, name = 'KOP Direction')
fig=PlotArrow(start=PTM[0,:], vector=VTM[0,:]*20, fig=fig, style='m-', head_size=0.5, name = 'Target Direction')
fig.show()

From the figure above, we can easily identify that the offset wells __#3,#4,#5,#6,#7__ are critical for anticollision constraints.

In [9]:
danger_indices=[2,3,4,5,6]

In [10]:
fig1=None
for well in danger_indices:
    fig1=PlotSurvey(input_data["SURVEY"][well], fig1, show=0)
     
# # plot the PK and PT points in the fig
# fig1.add_trace(go.Scatter3d(x=PKM[:,0], y=PKM[:,1], z=PKM[:,2], 
#                            name='KOP',
#                            mode='markers',
#                            marker=dict(size=3, color='blue')))
# fig1.add_trace(go.Scatter3d(x=PTM[:,0], y=PTM[:,1], z=PTM[:,2], 
#                            name='Target',
#                            mode='markers',
#                            marker=dict(size=3, color='magenta')))

# plot an arrow of the VT, start at PT
fig1=PlotArrow(start=PKM[0,:], vector=VKM[0,:]*300, fig=fig1, style='b-', head_size=0.5, name = 'KOP Direction')
fig1=PlotArrow(start=PTM[0,:], vector=VTM[0,:]*20, fig=fig1, style='m-', head_size=0.5, name = 'Target Direction')
fig1.show()

### 1.4.2 prepare the 'anticol_con' dictionary

The minimum safe distance to the wellbore can be a function related to the measured depth ($md$), or simply a scalar.  
The safe distance is determined by a lot of factors, such as the drilling(survey) equipment accuracy, drilling crews' operation, underground situation, etc.  
It's a concept that covers the EOU(ellipsoid of uncertainty) of the position survey.

In this example, we show the ability of handling a user-defined function of $md$, rather than a simple fix value:  
Offset wells safe distance: $2.5+(md/300)^2$  
New well safe distance: $2.5+(md/500)^2$

In [11]:
# initialize keys in `anticol_con`
Nodes_offset_list=[]
safeDist_offset_f_list=[]
safeDist_new_f_list=[]
opt_factor_list=[]
SF_list=[]

# for well in range(len(input_data["SURVEY"])):
for well in [2,3,4,5,6]:
    x = np.array(input_data["SURVEY"][well]['X']) # array of X coordinates
    y = np.array(input_data["SURVEY"][well]['Y']) # array of Y coordinates
    z = np.array(input_data["SURVEY"][well]['Z']) # array of Z coordinates
    MD = np.array(input_data["SURVEY"][well]['MD']) # array of MD
    Nodes_offset_list.append(np.array([x, y, z, MD]).T) # append a 3D array of [x,y,z] coordinates

    # safe distance for offset wells, a list of strings, the string is a function of md
    safeDist_offset_f_list.append("2.5+(md/300)**2") 
    # safe distance for new well, a list of strings, corresponding to offset wells
    safeDist_new_f_list.append("2.5+(md/500)**2") 

    opt_factor_list.append(2)
    SF_list.append(1)

anticol_con={
    "Nodes_offset_list": Nodes_offset_list,
    "safeDist_offset_f_list": safeDist_offset_f_list,
    "safeDist_new_f_list": safeDist_new_f_list,
    "opt_factor_list": opt_factor_list,
    "SF_list": SF_list,
}

## 1.5 [Optional] Objective(cost) function
User-defined cost function for the trajectory, default is the length of the trajectory (KOP->Target).


In [ ]:
ObjM=None


## 1.6 [Optional] Computational parameters 
Normally, you do not need to bother yourself about these paramters. API will automatically apply default values.

In [13]:
# discretized measured depth for trajectory data
MD_intervalM= None, # shape (n, ), default: 30m

In [14]:
# plot canvas
XRange= None, # shape (2, ), default: [min(X) of given points(PKM, PTM) - max(rM),  max(X) of given points(PKM, PTM) + max(rM)]
YRange= None, # shape (2, )

In [15]:
# grid resolution for cost contour
resolution= None, # scalar, default is 100. This affects the computational time significantly because of the cost contour's grid node values.

In [16]:
# radius of the cost contour for each well
cst_radiusM= None, # shape (n, ), default is 3000

# 2. Use the API to do the computation

## 2.1 save the input data to JSON

In [17]:
# save the input dataset with anticollision
filepath_anticollision="input_anticollision.json"

input_anticollision=input2json(n,                
        PTM,
        VTM,
        PKM,
        VKM,
        DLSM,
        ObjM=ObjM,
        
        anticol_con=anticol_con, # anticollision information
        
        MD_intervalM=MD_intervalM,
        XRange=XRange,
        YRange=YRange,
        resolution=resolution,
        cst_radiusM=cst_radiusM,
        
        filepath=filepath_anticollision)

# save the input dataset without anticollision, for comparison
filepath_free="input_free.json"
input_free=input2json(n,                
        PTM,
        VTM,
        PKM,
        VKM,
        DLSM,
        ObjM=ObjM,
        
        MD_intervalM=MD_intervalM,
        XRange=XRange,
        YRange=YRange,
        resolution=resolution,
        cst_radiusM=cst_radiusM,
        
        filepath=filepath_free)

=====file input_anticollision.json written successfully=====
=====file input_free.json written successfully=====


## 2.2 prepare the parameters for calling API

If you have already saved the Json input before, load it directly.

In [18]:
# # load saved input data (json) for computation
# with open(input_anticollision) as json_file:
#     input_anticollision = json.load(json_file)
# with open(filepath_free) as json_file:
#     input_free = json.load(json_file)

Define the output data file path for saving the response from the API server

In [19]:
# filepath for saving the output
output_free_filepath="output_free.json" 
output_anticollision_filepath="output_anticollision.json" 

## 2.3 fetch the response from the API

**The test server has very limited computing power**, it takes a long time to deal with anticollision.  
This case considering 5 offset wells takes about 4 minutes to get back the result.  
The result includes the wellbore trajectory specified by the `index`, default index=0.

In [20]:
tic=time.time()
output_anticollision=get_1well(input_anticollision,
                filepath=output_anticollision_filepath,
                )
toc=time.time()
print(f"Elapsed time: {toc-tic} seconds")

requesting from https://home-test.make234.com/api/v1/get_1well
Status code: 200
Response content has been saved to "output_anticollision.json"
Elapsed time: 225.65748810768127 seconds


Without anticollision, it only takes 4 seconds.

In [21]:
tic=time.time()
output_free=get_1well(input_free,
                filepath=output_free_filepath,
                )
toc=time.time()
print(f"Elapsed time: {toc-tic} seconds")

requesting from https://home-test.make234.com/api/v1/get_1well
Status code: 200
Response content has been saved to "output_free.json"
Elapsed time: 1.3066589832305908 seconds


# 3. Visualize the results

## 3.1 load and check the data

In [22]:
# # Load the output JSON file
# with open(output_anticollision_filepath, "r") as file:
#     output_anticollision = json.load(file)

In [23]:
# Extract the trajectory data 
wellbore_anticollision = output_anticollision["data"]["Trajectories"][0]


In [24]:
wellbore_free = output_free["data"]["Trajectories"][0]

## 3.2 3D Visualization
Plot the wellbore trajectories and the cost contour for the site.  
The cost contour indicates the total cost to reach all the targets while placing the drill site at the given [X, Y] location.  
If the cost contour at a given point is empty, then it means the site located at this point cannot reach at least one of the targets under the given constraints.

In [25]:
try:
    fig_result=copy.deepcopy(fig)
except:
    fig_result=None

In [26]:
# Plot anticollision wellbore trajectory
fig_result=PlotSurvey(wellbore_anticollision, # trajectory data
                fig=fig_result, # figure handle
                style='b-', # line style for the trajectory
                name=f'New Well anticollision', # name of the trajectory in the plot
                show=0) # wait for plottiing all trajectories

# show the figure
fig_result.show()

report the dangerous points

In [ ]:
# collect the output_anticollision["data"]["AnticolReport"] into a pandas dataframe
df_anticollision=pd.DataFrame(output_anticollision["data"]["AnticolReport"])

# add a column at the start
df_anticollision.insert(0, "Offset_Well_No", [danger_indices[i]+1 for i in df_anticollision.index])

# show the table
df_anticollision

,Offset_Well_No,Danger_Distance,Minimum_Safe_Distance,Danger_Point_Offset,Danger_Point_New
0,3,324.790906,46.137172,"[-1431.8815000000177, -5384.337100000121, -152...","[-1125.731269982479, -5277.767454898944, -1549..."
1,4,161.477909,53.791976,"[-938.4858552533645, -5422.325033347138, -1635...","[-1088.0044459069, -5413.692871621839, -1695.6..."
2,5,86.691211,60.287136,"[-988.7536600000458, -5526.05709999986, -1753....","[-1062.853910585332, -5504.3073635159735, -179..."
3,6,80.884562,57.310782,"[-1160.7464300000574, -5436.2300999998115, -17...","[-1082.138361778686, -5434.827699405425, -1718..."
4,7,66.613373,66.618230,"[-979.1500500000548, -5598.7610999997705, -185...","[-1039.6426753700875, -5587.93477972691, -1882..."


show the simple free trajectory without considering anticollision constraints

In [27]:
# Plot free wellbore trajectory for comparison
fig_result=PlotSurvey(wellbore_free, # trajectory data
                fig=fig_result, # figure handle
                style='r-', # line style for the trajectory
                name=f'New Well free', # name of the trajectory in the plot
                show=0) # wait for plottiing all trajectories

# show the figure
fig_result.show()

save the interactive 3D figure

In [28]:
figpath="../../../docs/figure_1well_ex2.html"
fig_result.update_layout(
            autosize=True,  # auto resize the fig1 in webbrowers
            width=None,
            height=None,
            margin=dict(l=0, r=0, t=0, b=0),)
fig_result.write_html(figpath, full_html=True, div_id="plotly-div", config={"responsive": True})
print(f"figure saved to file: \"{figpath}\"")

figure saved to file: "../../../docs/figure_1well_ex2.html"
